In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join("../code"))
sys.path.insert(0, module_path)

In [2]:
from pprint import pprint

import cnwi
import ee
import eelib
import geemap
import geopandas as gpd

from pipelines import factories

# Set inputs up

In [3]:
gdf = gpd.read_file(
    filename='../data/willistonA_no_floodplain.shp',
    driver='ESRI Shapefile'
)

# re name CDesc to land_cover and drop all other non geometry columns
gdf = gdf[['cDesc', 'geometry']]
gdf.columns = ['land_cover', 'geometry']

training_data = ee.FeatureCollection.from_dataframe(gdf)
pprint(training_data.first().getInfo())

viewport = ee.FeatureCollection.from_file(
    filename="../results/phase02/000-data/viewport.shp",
    layer=None,
    driver='ESRI Shapefile'
).geometry()

data_cube_collection = factories.DatacubeCollection("projects/fpca-336015/assets/williston-cba")
s1_system_ids = "../data/systems.txt"

{'geometry': {'coordinates': [-122.09316427210513, 54.62236146027936],
              'type': 'Point'},
 'id': '0',
 'properties': {'land_cover': 'Bog'},
 'type': 'Feature'}


In [4]:
datacube = cnwi.run_datacube_lsc(
    viewport=viewport,
    training_data=training_data,
    system_txt = s1_system_ids,
    dc_collection=data_cube_collection,
    phase='Phase04'
)

datacube[0].logging()
datacube[0].exportToCloud()